In [1]:
#-------------------------------------------------------------------------------------------
#
#  Moteur de recherche et de recommendation - Open Food Data
#
#-------------------------------------------------------------------------------------------
#
# Importing the libraries
#
#-------------------------------------------------------------------------------------------

import pandas as pd
from IPython.display import Image, HTML
import matplotlib.pyplot as plt
#
#    from wordcloud import WordCloud, STOPWORDS  - needs Python 3.7
#
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jtorr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#---------------------------------------------------------------------------------------------------
#
#  Fonction clean_chars qui remplace des characters dans un dataframe par un autre
#  utilisée pour remplacer des lettres avec des signes diacritiques
#
#  strng1: string contenant les characters à remplacer
#  strng2: string contenant les characters à chercher
#  strng3: string contenant les characters remplaçants
#
#  CHANGE DANS STRNG1 TOUS LES CHARACTERS DANS STRING2 PAR LE(s) CHARACTER(S) DANS STRING3
#
#---------------------------------------------------------------------------------------------------

def clean_chars(strng1, strng2, strng3):
    for c in strng2:
        if (c in strng1):
            strng1 = strng1.replace(c,strng3)
    return strng1


In [3]:
#-------------------------------------------------------------------------------------------
#
# Importing the dataset
#
#-------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------
# 
# Chargement du fichier fr.openfoodfacts.products.csv
# Le fichier est placé dans le repertoire courant. 
# Le fichier contient uniquement des produits vendus en France
#
#----------------------------------------------------------------------------------------

food_data = pd.read_csv('fr.openfoodfacts.products.csv', sep=',',
                        usecols=['code',
                                 'product_name',
                                 'brands',
                                 'categories',
                                 'ingredients_text',
                                 'allergens',
                                 'traces',
                                 'nutrition_grade_fr',
                                 'energy_100g',
                                 'sugars_100g',
                                 'fiber_100g',
                                 'proteins_100g',
                                 'sodium_100g',
                                 'alcohol_100g',
                                 'fruits-vegetables-nuts_100g',
                                 'nutrition-score-fr_100g'
                                ],
                        dtype = {'code': str,
                                 'product_name': str,
                                 'brands': str,
                                 'categories': str, 
                                 'ingredients_text': str,
                                 'allergens': str,
                                 'traces': str,
                                 'nutrition_grade_fr': str,
                                 'energy_100g': float,
                                 'sugars_100g': float,
                                 'fiber_100g': float,
                                 'proteins_100g': float,
                                 'sodium_100g': float,
                                 'alcohol_100g': float,
                                 'fruits-vegetables-nuts_100g': float, 
                                 'nutrition-score-fr_100g': float
                                }
                       )
#
food_data.shape

(123145, 16)

In [4]:
#-------------------------------------------------------------------------------------------
#
#  Exploring the alphabetic fields the dataset. 
#
#  code
#  product_name
#  brands
#  categories
#  ingredients_text
#  allergens
#  traces
#
#-------------------------------------------------------------------------------------------

col_names = ['product_name','brands','categories','ingredients_text','allergens']

for col in col_names:
    print(food_data[col].head(n=10),'\n')


0                   Farine de blé noir
1                  Naturablue original
2                        Filet de bœuf
3                 Naturakrill original
4                       Lion Peanut x2
5                              Twix x2
6                       Pack de 2 Twix
7                     lentilles vertes
8                            Root Beer
9    Biscuits sablés fourrage au cacao
Name: product_name, dtype: object 

0     Ferme t'y R'nao
1         Natura4ever
2                 NaN
3         Natura4ever
4                 NaN
5                 NaN
6      Twix, Lundberg
7    Bertrand Lejeune
8                 A&W
9           St Michel
Name: brands, dtype: object 

0                                                  NaN
1                                                  NaN
2                                        Filet de bœuf
3                                                  NaN
4                                                  NaN
5                                                

In [5]:
#-------------------------------------------------------------------------------------------
#
#  Compter les nulls par colonne 
#
#-------------------------------------------------------------------------------------------

food_data.isnull().sum(axis = 0)

code                                0
product_name                        2
brands                           6343
categories                      46635
ingredients_text                36383
allergens                       92414
traces                         100626
nutrition_grade_fr              30118
energy_100g                     26703
sugars_100g                         0
fiber_100g                          0
proteins_100g                       0
sodium_100g                     28512
alcohol_100g                   120391
fruits-vegetables-nuts_100g    120002
nutrition-score-fr_100g         30118
dtype: int64

In [6]:
#-------------------------------------------------------------------------------------------
#
#  J'élimine les produits avec product_name null
#
#-------------------------------------------------------------------------------------------

food_data = food_data.dropna(subset=['product_name'])

food_data.shape


(123143, 16)

In [7]:
food_data.isnull().sum(axis = 0)

code                                0
product_name                        0
brands                           6342
categories                      46633
ingredients_text                36381
allergens                       92412
traces                         100624
nutrition_grade_fr              30116
energy_100g                     26701
sugars_100g                         0
fiber_100g                          0
proteins_100g                       0
sodium_100g                     28510
alcohol_100g                   120389
fruits-vegetables-nuts_100g    120000
nutrition-score-fr_100g         30116
dtype: int64

In [8]:
#-------------------------------------------------------------------------------------------
#
#  Nettoyage de champs alphanmériques #1
#
#
#  Remplacemente des nulls par des blancs
#
#-------------------------------------------------------------------------------------------

col_names = ['brands','categories','ingredients_text','allergens','traces','nutrition_grade_fr']

for col in col_names:
    food_data[col] = food_data[col].fillna('') 

for col in col_names:
    food_data[col] = food_data[col].str.strip() 

0                                                          
1                                                          
2                                             Filet de bœuf
3                                                          
4                                                          
                                ...                        
123140                                           thés verts
123141                                                     
123142    Viandes,Produits à tartiner,Charcuteries,Produ...
123143                                                     
123144                                                     
Name: categories, Length: 123143, dtype: object


In [9]:
food_data.isnull().sum(axis = 0)

code                                0
product_name                        0
brands                              0
categories                          0
ingredients_text                    0
allergens                           0
traces                              0
nutrition_grade_fr                  0
energy_100g                     26701
sugars_100g                         0
fiber_100g                          0
proteins_100g                       0
sodium_100g                     28510
alcohol_100g                   120389
fruits-vegetables-nuts_100g    120000
nutrition-score-fr_100g         30116
dtype: int64

In [10]:
#-------------------------------------------------------------------------------------------
#
#  Nettoyage de champs alphanumériques #2
#
#  Pour la suite, on va créer un nouveau dataframe "corpus", contenant le 'code' 
#  de produit dans une colonne et une autre contenant, et une concatenation de 
#  tous les champs alphabétiques sauf le nutritionscore_grade.
#
#  Sur ce nouveau dataframe, on va procéder au nettoyage des charactères spéciaux, 
#  de signes diacritiques, des signes orthographiques, etc., avec un '-' de separation.
#
#  Ce sera la dataframe pour le moteur de recherche du système de recommendation.
#
#-------------------------------------------------------------------------------------------

col_names = ['code', 'description', 'description_bow']

corpus = pd.DataFrame(columns = col_names)

col_names = ['brands', 'categories', 'ingredients_text', 'allergens', 'traces']

corpus['code'] = food_data['code']

corpus['description'] = food_data[col_names].agg('-'.join, axis=1)

corpus.shape


(123143, 3)

In [11]:
col_names = ['code','description', 'description_bow']

for col in col_names:
    print(corpus[col].head(n=20),'\n')


0     0000000003087
1     0000000020114
2     0000000024600
3     0000000030113
4     0000000036252
5     0000000039259
6     0000000039529
7     0000005200016
8     0000007020254
9     0000007730009
10    0000009336247
11    0000010068175
12    0000010090206
13    0000010127735
14    0000010179413
15    0000010187319
16    0000010207260
17    0000020004552
18    0000030053014
19    0000040144078
Name: code, dtype: object 

0                                   Ferme t'y R'nao----
1                                       Natura4ever----
2                                     -Filet de bœuf---
3                                       Natura4ever----
4                                                  ----
5                                                  ----
6                                    Twix, Lundberg----
7     Bertrand Lejeune-Aliments et boissons à base d...
8     A&W-Boissons,Boissons gazeuses,Sodas,Boissons ...
9     St Michel-Snacks sucrés,Biscuits et gâteaux,Bi...
10       Ner

In [12]:
#-------------------------------------------------------------------------------------------
#
#  Nettoyage de champs alphanmériques #2
#
#
#  Remplacement des charactères spéciaux par des espaces blancs
#
#-------------------------------------------------------------------------------------------

sp_ch = ['º','$', '%', '&','(',')','=','*','?','^','+','*','¨',';',':','`','-','_','/','´',
         '_',',','ª','¿','¡','Ç','{','}','[',']','<','>','\'','´']

for sc in sp_ch:
    corpus['description'] = corpus['description'].str.replace(sc,' ')

print('Ready')

Ready


In [13]:
#-------------------------------------------------------------------------------------------
#
#  Nettoyage de champs alphanmériques #3
#
#
#  Remplacement des charactères avec des signes diacritiques par des charactères simplifiés
# 
#  Le fichier alphabet.csv contient tous les lettres de l'alphabet avec toutes les
#
#  possibilités de signes diacritiques et son remplacement correspondant dans chaque cas
#
#-------------------------------------------------------------------------------------------
#
#  alphabet.csv contient deux colonnes de characters
#
#-------------------------------------------------------------------------------------------

alphabet = pd.DataFrame(['initial','final'])

alphabet = pd.read_csv('alphabet.csv',
                     sep=',',
                     usecols=['initial', 'final'],
                     dtype = {'initial': str, 'final': str}
                    )

#-------------------------------------------------------------------------------------------
#
#  Remplacemet de tous les characters avec des signes diacritiques, lettres doubles, etc.
#
#-------------------------------------------------------------------------------------------

for k in range(0,len(corpus)):
    strng1 = corpus.iloc[k,1]
    for i in range(0,len(alphabet)):
        strng2 = alphabet.iloc[i,0]
        strng3 = alphabet.iloc[i,1]
        strng1 = clean_chars(strng1, strng2, strng3)
    corpus.iloc[k,1] = strng1
    
print('Ready')


Ready


In [20]:
print(corpus['description'].head(n=30),'\n')

0                                   ferme t y r nao    
1                                       natura4ever    
2                                     filet de boeuf   
3                                       natura4ever    
4                                                      
5                                                      
6                                    twix  lundberg    
7     bertrand lejeune aliments et boissons a base d...
8     a w boissons boissons gazeuses sodas boissons ...
9     st michel snacks sucres biscuits et gateaux bi...
10      nerds snacks sucres confiseries bonbons   Oeufs
11    alice delice en beverages the noir aromatise a...
12    alice delice aliments et boissons a base de ve...
13    alice delice sirops sirops pour ganache sirop ...
14    alice delice  farine de ble  gluten   sucre de...
15                                     alice delice    
16                                     alice delice    
17    union des vignerons des cotes du rhone boi

In [31]:

for i in range(0, len(corpus)):
    corpus.iloc[i,1] = corpus.iloc[i,1].strip()
    corpus.iloc[i,2] = corpus.iloc[i,1].split(' ')


In [32]:

corpus['description'].map(len)

# Fin de 8 mayo 2020 . continuaré el 9 de mayo

0          15
1          11
2          14
3          11
4           0
         ... 
123140    106
123141      0
123142    238
123143     12
123144      5
Name: description, Length: 123143, dtype: int64

In [21]:
#-------------------------------------------------------------------------------------------
#   Dataframe qui contiendra tous les possibles réponses à la questions del utilisateur
#-------------------------------------------------------------------------------------------

col_names = ['code', 'tf_idf', 'cos_sim']

corpus_vector = pd.DataFrame(columns = col_names)

corpus_vector.columns

Index(['code', 'tf_idf', 'cos_sim'], dtype='object')

In [22]:
#-------------------------------------------------------------------------------------------
#
#  L'utilisateur introduit sa requête. Elle sera stockée dans la variable "user_input"
#
#-------------------------------------------------------------------------------------------

user_input = "farine de blé française où étrangère & % $"

#-------------------------------------------------------------------------------------------
#
#  user_input est nettoyé et normalisé tel que la description du corpus
#
#-------------------------------------------------------------------------------------------

sp_ch = ['º','$', '%', '&','(',')','=','*','?','^','+','*','¨',';',':','`','-','_','/','´',
         '_',',','ª','¿','¡','Ç','{','}','[',']','<','>','\'','´']

for sc in sp_ch:
    user_input = user_input.replace(sc,' ')

for i in range(0,len(alphabet)):
    strng2 = alphabet.iloc[i,0]
    strng3 = alphabet.iloc[i,1]
    user_input = clean_chars(user_input, strng2, strng3)

print(user_input)

#
# Les espaces blanc sont eliminés au début et à la fin
#

user_input = user_input.strip()

#
# user_input est transformé en "sac de paroles (bag of words)"
#
user_input_BoW = user_input.split(' ')

print(user_input_BoW)

farine de ble americain ou etrangere      
['farine', 'de', 'ble', 'americain', 'ou', 'etrangere']


In [ ]:
#------------------------------------------------------------------------------------------
#
# Initialize CountVectorizer
#
# In order to start using TfidfTransformer you will first have to create a CountVectorizer 
# to count the number of words (term frequency), limit your vocabulary size, 
# apply stop words and etc. The code below does just that.
#
#-------------------------------------------------------------------------------------------
#   instantiate CountVectorizer()
#-------------------------------------------------------------------------------------------

cv = CountVectorizer()
 
# this steps generates word counts for the words user_input
word_count_vector=cv.fit_transform(corpus['description'])

word_count_vector.shape

In [52]:
word_count_vector.iterrows()

AttributeError: iterrows not found